In [ ]:
#Install required dependencies

%pip install python-terrier==0.10.0  ir_measures ir_datasets fast-forward-indexes==0.2.0

In [ ]:
import pyterrier as pt
from pathlib import Path
from fast_forward.encoder import TCTColBERTQueryEncoder, TCTColBERTDocumentEncoder
from fast_forward.util.pyterrier import FFScore, FFInterpolate
from fast_forward import OnDiskIndex, Mode, Ranking
from pyterrier.measures import RR, nDCG, MAP, AP, R
import pandas as pd
import torch

In [ ]:
if not pt.started():
    pt.init()

dataset_name = 'irds:beir/fiqa'
dataset = pt.get_dataset(dataset_name)
#Create lexical index for BM25 - stores index files locally
indexer = pt.IterDictIndexer(
str(Path.cwd()),
meta={'docno': 7})
indexer.index(dataset.get_corpus_iter(), fields=['text'])
#Create index from local directory
index = pt.IndexFactory.of(str(Path.cwd()), memory=True)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [ ]:
devset_name = 'irds:beir/fiqa/dev'
devset = pt.get_dataset(devset_name)
#Retrieve 1000 documents using BM25 and write the results to file
candidates = (bm25 % 1000)(devset.get_topics())
pt.io.write_results(candidates,'trec-run-bm25_fiqa-dev.txt')

In [ ]:
#TCT ColBERT Hyperparameter tuning

def docs_iter(dataset):
    for d in dataset.get_corpus_iter():
        yield {"doc_id": d["docno"], "text": d["text"]}

torch.cuda.is_available()

q_encoder = TCTColBERTQueryEncoder("castorini/tct_colbert-msmarco")
d_encoder = TCTColBERTDocumentEncoder(
    "castorini/tct_colbert-msmarco",
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)
#Create Fast Forward index
ff_index = OnDiskIndex(Path("ffindex_fiqa_tct.h5"), dim=768, query_encoder=q_encoder, mode=Mode.MAXP)
#Load index from disk 
# ff_index = OnDiskIndex.load(
#     Path("/projects/neural_ranking/ff_indexes/tct_colbert/ff_msmarco-v1-passage.tct_colbert.h5"),  query_encoder=q_encoder, mode=Mode.MAXP
# )
ff_index = ff_index.to_memory()
ff_score = FFScore(ff_index)
re_ranked = ff_score(candidates)
ff_int = FFInterpolate(alpha=0.5)
#Hyperparameter tuning to find alpha for interpolation
df: pd.DataFrame = pt.GridSearch(bm25 % 1000 >> ff_score >> ff_int,
{ff_int: {"alpha": [0.05, 0.1, 0.25,0.5, 0.75,0.9]}},
devset.get_topics(),
devset.get_qrels(),
"ndcg_cut_10",
verbose=True,)

In [ ]:
#Interpolation-based re-ranking results
#best alpha
print(ff_int.alpha)
testset_name = 'irds:beir/fiqa/test'
testset = pt.get_dataset(testset_name)
#BM25 first-stage retrieval
candidates = (bm25 % 1000)(testset.get_topics())
pt.io.write_results(candidates,'trec-run-bm25_fiqa-test-0.1.txt')
re_ranked = ff_score(candidates)
ff_int = FFInterpolate(alpha=ff_int.alpha)
# BM25-> TCT-ColBERT interpolation-based re-ranking
res = ff_int(re_ranked)
new_df = res[['qid', 'docno', 'score']].rename(columns={'qid': 'q_id', 'docno': 'id'})
r = Ranking(new_df)
#Save FF results
r.save(Path("trec-run-ff-bm25-tct-fiqa-test-0.1.txt"))
#Conduct experiment + calculate the metrics
df: pd.DataFrame = pt.Experiment(
[bm25, bm25 % 1000 >> ff_score >> ff_int],
testset.get_topics(),
testset.get_qrels(),
eval_metrics=[RR @ 10, nDCG @ 10, R@ 100],
names=["BM25", "BM25 to TCT"],
)